In [14]:
import os
import sys
import gc
import pickle
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
from typing import *
from tqdm.notebook import tqdm
from pathlib import Path
from matplotlib import pyplot as plt

pd.set_option('max_columns', 50)
pd.set_option('max_rows', 200)
warnings.simplefilter('ignore')
sns.set()

In [15]:
base_dir = Path().resolve()
sys.path.append(str(base_dir / '../'))

from utils.preprocess import *
from utils.model import *
from utils.train import *
from utils.eval import *
from utils.ensemble import *

In [16]:
from dataclasses import dataclass, field, asdict
import yaml


@dataclass
class Config:
    debug: bool = False
    outdir: str = "results00"
    device: str = "cuda:2"
    device_id: int = 2
    # Data config
    n_splits: int = 5
    imgdir_name: str = "../../data/VinBigData/png[tmp]"
    datadir_name: str = '../../data/VinBigData'
        
    img_sizes: List[int] = field(default_factory=lambda: [])
    batch_size: int = 25
    test_aug_kwargs: Dict[str, Dict[str, Any]] = field(default_factory=lambda: {})
        
    def update(self, param_dict: Dict) -> "Config":
        # Overwrite by `param_dict`
        for key, value in param_dict.items():
            if not hasattr(self, key):
                raise ValueError(f"[ERROR] Unexpected key for flag = {key}")
            setattr(self, key, value)
        return self

In [17]:
config = Config().update({
    'debug': False,
    'batch_size': 25,
    'img_sizes': [256, 512, 1024],
    "test_aug_kwargs": {
        "Normalize": {"mean": (0.485, 0.456, 0.406), "std": (0.229, 0.224, 0.225)}
    },
})

In [18]:
classes_nms = [
    "Aortic enlargement",
    "Atelectasis",
    "Calcification",
    "Cardiomegaly",
    "Consolidation",
    "ILD",
    "Infiltration",
    "Lung Opacity",
    "Nodule/Mass",
    "Other lesion",
    "Pleural effusion",
    "Pleural thickening",
    "Pneumothorax",
    "Pulmonary fibrosis",
    'No Finding'
]
classes_dict = {index + 1: class_name  for index, class_name in enumerate(classes_nms)}

In [19]:
def load_train_data(filepath: str, meta_filepath: str, img_size: int) -> pd.DataFrame:
    train = pd.read_csv(filepath)
    train.fillna(0, inplace=True)
    train.loc[train["class_id"] == 14, ['x_max', 'y_max']] = 1.0
    
    train_meta = pd.read_csv(meta_filepath)
    
    train = pd.merge(train, train_meta, how='left', on='image_id')
    
    train[f'x_min_{img_size}'] = (img_size / train['dim1'] * train['x_min']).astype(int)
    train[f'x_max_{img_size}'] = (img_size / train['dim1'] * train['x_max']).astype(int)
    train[f'y_min_{img_size}'] = (img_size / train['dim0'] * train['y_min']).astype(int)
    train[f'y_max_{img_size}'] = (img_size / train['dim0'] * train['y_max']).astype(int)
    
    return train

In [20]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [27]:
def predict(device: str, outdir: Path, test_loader: DataLoader, image_size: int, test_meta: pd.DataFrame):
    # Predict
    device = torch.device(config.device)

    detector = ObjectDetector(model=load_faster_rcnn(num_classes=len(classes_nms) + 1), train_evaluator=None, valid_evaluator=None, outdir=outdir)
    detector.load_state_dict(torch.load(str(outdir / 'model_best.pt'), map_location=device))
    detector.to(device)
    detector.eval()

    preds_list = list()
    for batch in tqdm(test_loader):
        images, image_ids = batch
        images = list(image.to(device) for image in images)
        preds = detector(images)

        del images
        gc.collect()

        for i in range(len(preds)):
            preds[i] = {k: preds[i][k].detach().cpu().numpy() for k in preds[i].keys()}
            preds[i]['image_id'] = image_ids[i]
        preds_list += preds
    
    # to Original Image Size
    records = {'image_id': list(), 'objects': list()}

    for preds in tqdm(preds_list):
        image_id = preds['image_id']
        x_dicom = int(test_meta.loc[test_meta['image_id'] == image_id, 'dim1'])
        y_dicom = int(test_meta.loc[test_meta['image_id'] == image_id, 'dim0'])

        objects = list()
        for box, label, score in zip(preds['boxes'], preds['labels'], preds['scores']):
            box_ = [
                box[0] * (x_dicom / img_size), # x_min
                box[1] * (y_dicom / img_size), # y_min
                box[2] * (x_dicom / img_size), # x_max
                box[3] * (y_dicom / img_size), # y_max
            ]
            objects += [[label] + [score] + [box_]]

        records['image_id'] += [image_id]
        records['objects'] += [objects]

    return records

In [28]:
img_size = 1024
imgdir_name = config.imgdir_name.replace('[tmp]', str(img_size))
test_meta = pd.read_csv(str(base_dir / config.datadir_name / 'test_meta.csv'))
dataset_dicts_test = get_vinbigdata_dicts_test(
    base_dir / imgdir_name, 
    test_meta,
    test_data_type=f'png{img_size}',
    debug=config.debug
)

test_dataset = VinBigDataset(dataset_dicts=dataset_dicts_test, transform=Transform(config.test_aug_kwargs, train=False), train=False)
test_loader = DataLoader(
    test_dataset,
    batch_size=config.batch_size,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

pred_df = predict(device=config.device, outdir=base_dir / 'results00/fold-1', test_loader=test_loader, image_size=img_size, test_meta=test_meta)

Load from cache dataset_dicts_cache_test_png1024_debug0.pkl


  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

In [11]:
# preds_dfs = dict()

# for i, img_size in enumerate(config.img_sizes):
#     imgdir_name = config.imgdir_name.replace('[tmp]', str(img_size))
#     test_meta = pd.read_csv(str(base_dir / config.datadir_name / 'test_meta.csv'))
#     dataset_dicts_test = get_vinbigdata_dicts_test(
#         base_dir / imgdir_name, 
#         test_meta,
#         test_data_type=f'png{img_size}',
#         debug=config.debug
#     )
    
#     test_dataset = VinBigDataset(dataset_dicts=dataset_dicts_test, transform=Transform(config.test_aug_kwargs, train=False), train=False)
#     test_loader = DataLoader(
#         test_dataset,
#         batch_size=config.batch_size,
#         shuffle=False,
#         num_workers=4,
#         collate_fn=collate_fn
#     )
    
#     preds_dfs[img_size] = list()
#     for fold in range(1, config.n_splits + 1):
#         print(f'[{fold}/{config.n_splits}]')
#         preds_df = predict(
#             device=config.device, 
#             outdir=base_dir / f'results0{i}/fold-{fold}',
#             test_loader=test_loader,
#             image_size=img_size,
#             test_meta=test_meta
#         )
        
#         preds_dfs[img_size] += [preds_df]

# with open(str(base_dir / 'preds_dfs.pickle'), 'wb') as f:
#     pickle.dump(preds_dfs, f)

In [12]:
# preds = pickle.load(open(str(base_dir / 'preds_dfs.pickle'), 'rb'))

In [13]:
# def to_ensemble_format(obj: List[Any]) -> List[Any]:
#     """
#     obj: [class, score, [x_min, y_min, x_max, y_max]]
#     return: [x_center, y_center, x_width, y_height, class, score]
#     """
#     label, score, coco_box = obj
#     box = [
#         coco_box[0] + 0.5 * (coco_box[2] - coco_box[0]),
#         coco_box[1] + 0.5 * (coco_box[3] - coco_box[1]),
#         coco_box[2] - coco_box[0],
#         coco_box[3] - coco_box[1]
#     ]
#     return box + [label] + [score]


# def to_coco_format(obj: List[Any]) -> List[Any]:
#     x_center, y_center, x_width, y_height, label, score = obj
#     box = [
#         x_center - 0.5 * x_width,
#         y_center - 0.5 * y_height,
#         x_center + 0.5 * x_width,
#         y_center + 0.5 * y_height
#     ]
#     return [label] + [score] + box

In [14]:
# test_preds = list()
# for img_size in config.img_sizes:
#     for fold in range(config.n_splits):
#         model_pred = list()
#         for image_id, objects in zip(preds[img_size][fold]['image_id'], preds[img_size][fold]['objects']):
#             model_pred += [list(map(to_ensemble_format, objects))]
#         test_preds += [model_pred]

In [15]:
# records = {'image_id': list(), 'PredictionString': list()}
# for i, image_id in tqdm(enumerate(preds[256][0]['image_id'])):
#     dets = list()
#     for model_pred in test_preds:
#         dets += [model_pred[i]]
#     ensembled = GeneralEnsemble(dets)
#     ensembled = list(map(to_coco_format, ensembled))
    
#     records['image_id'] += [image_id]
#     records['PredictionString'] += [' '.join(list(map(str, sum(ensembled, []))))]

# preds_df = pd.DataFrame(records)
# pickle.dump(preds_df, open(str(base_dir / 'preds_ensemble.pickle'), 'wb'))

In [16]:
# preds_df = pickle.load(open(str(base_dir / 'preds_ensemble.pickle'), 'rb'))

In [32]:
from ensemble_boxes import *
from copy import deepcopy

IOU_THR = 0.4

records = {'image_id': list(), 'PredictionString': list()}

for image_id, objs, (height, width) in tqdm(zip(pred_df['image_id'], pred_df['objects'], test_meta[['dim0', 'dim1']].values)):

    pred_str = list()
    
    for cls_id in range(0, 14):
        cls_labels, cls_scores, cls_boxes = list(), list(), list()

        for obj in objs:
            label_, score, box = obj
            label = label_ - 1
            if label == cls_id:
                cls_labels += [label]
                cls_scores += [score]
                cls_boxes += [[
                    float(np.clip(box[0] / width, 0, 1)),
                    float(np.clip(box[1] / height, 0, 1)),
                    float(np.clip(box[2] / width, 0, 1)),
                    float(np.clip(box[3] / height, 0, 1))
                ]]

        if len(cls_labels) > 0:
            cls_boxes_, cls_scores_, cls_labels_ = nms([deepcopy(cls_boxes)], [deepcopy(cls_scores)], [deepcopy(cls_labels)], weights=None, iou_thr=IOU_THR)

            for cb, cs, cl in zip(cls_boxes_, cls_scores_, cls_labels_):
                pred_str += [str(cl)] + [str(cs)] + np.array([
                    cb[0] * width,
                    cb[1] * height,
                    cb[2] * width,
                    cb[3] * height
                ]).astype(str).tolist()
    
    # No Finding
    nf_scores = [obj[1] for obj in objs if obj[0] == 15]
    if len(nf_scores):
        pred_str += [str(14)] + [str(np.mean(nf_scores))] + np.array([0, 0, 1, 1]).astype(str).tolist()
    
    records['image_id'] += [image_id]
    records['PredictionString'] += [' '.join(pred_str)]

preds_df = pd.DataFrame(records)

0it [00:00, ?it/s]

In [35]:
# records = {'image_id': list(), 'PredictionString': list()}

# for image_id, objs in zip(pred_df['image_id'], pred_df['objects']):
#     records['image_id'] += [image_id]
#     objs_str = list()
#     for label, score, box in objs:
#         objs_str += [str(label - 1)] + [str(score)] + np.array(box).astype(str).tolist()  # background (id = 0)
    
#     records['PredictionString'] += [' '.join(objs_str)]

# preds_df = pd.DataFrame(records)

In [42]:
pred_test_2class = pd.read_csv(str(base_dir / 'pfn_copy_test_pred.csv'))
pred_test_2class

,image_id,class0,class1
0,8dec5497ecc246766acfba5a4be4e619,0.999985,0.000015
1,287422bed1d9d153387361889619abed,0.049089,0.950911
2,1d12b94b7acbeadef7d7700b50aa90d4,0.991706,0.008294
3,6b872791e23742f6c33a08fc24f77365,0.788935,0.211065
4,d0d2addff91ad7beb1d92126ff74d621,0.995391,0.004608
...,...,...,...
2995,78b44b96b121d6075d7ae27135278e03,0.999924,0.000076
2996,afee8ff90f29b8827d0eb78774d25324,0.999992,0.000008
2997,6e07fab2014be723250f7897ab6e3df2,0.996261,0.003739
2998,690bb572300ef08bbbb7ebf4196099cf,0.939969,0.060031


In [43]:
submission = pd.merge(pred_test_2class, preds_df, how='left', on='image_id')
submission

,image_id,class0,class1,PredictionString
0,8dec5497ecc246766acfba5a4be4e619,0.999985,0.000015,0 0.07871365 1054.7661862373352 620.8049368858...
1,287422bed1d9d153387361889619abed,0.049089,0.950911,0 0.5376988 1181.5101127624512 623.65135174989...
2,1d12b94b7acbeadef7d7700b50aa90d4,0.991706,0.008294,0 0.15403092 1183.1813017129898 917.8276730775...
3,6b872791e23742f6c33a08fc24f77365,0.788935,0.211065,3 0.08234966 720.6687622070312 1553.2708168029...
4,d0d2addff91ad7beb1d92126ff74d621,0.995391,0.004608,0 0.25614455 1450.710283279419 856.62181365489...
...,...,...,...,...
2995,78b44b96b121d6075d7ae27135278e03,0.999924,0.000076,14 0.9945011 0 0 1 1
2996,afee8ff90f29b8827d0eb78774d25324,0.999992,0.000008,14 0.9953649 0 0 1 1
2997,6e07fab2014be723250f7897ab6e3df2,0.996261,0.003739,0 0.56051755 1653.393310546875 820.89193725585...
2998,690bb572300ef08bbbb7ebf4196099cf,0.939969,0.060031,0 0.1009274 1122.0880651474 747.6574833393097 ...


In [44]:
low_threshold = 0
high_threshold = 0.976

for i in tqdm(submission.index):
    p0 = submission.loc[i, 'class0']
    
    if p0 < low_threshold:
        pass
    elif low_threshold <= p0 and p0 < high_threshold:
        submission.loc[i, 'PredictionString'] += f" 14 {p0} 0 0 1 1"
    else:
        submission.loc[i, 'PredictionString'] = '14 1 0 0 1 1'

  0%|          | 0/3000 [00:00<?, ?it/s]

In [45]:
submission = submission.drop(columns=['class0', 'class1'])
submission.to_csv(str(base_dir / 'submission.csv'), index=False)

In [41]:
submission

,image_id,PredictionString
0,8dec5497ecc246766acfba5a4be4e619,0 0.07871365 1054.7661862373352 620.8049368858...
1,287422bed1d9d153387361889619abed,0 0.5376988 1181.5101127624512 623.65135174989...
2,1d12b94b7acbeadef7d7700b50aa90d4,0 0.15403092 1183.1813017129898 917.8276730775...
3,6b872791e23742f6c33a08fc24f77365,3 0.08234966 720.6687622070312 1553.2708168029...
4,d0d2addff91ad7beb1d92126ff74d621,0 0.25614455 1450.710283279419 856.62181365489...
...,...,...
2995,78b44b96b121d6075d7ae27135278e03,14 0.9945011 0 0 1 1
2996,afee8ff90f29b8827d0eb78774d25324,14 0.9953649 0 0 1 1
2997,6e07fab2014be723250f7897ab6e3df2,0 0.56051755 1653.393310546875 820.89193725585...
2998,690bb572300ef08bbbb7ebf4196099cf,0 0.1009274 1122.0880651474 747.6574833393097 ...
